# Projeto 1 - Ciência dos Dados

Nome: Luis Antonio Bordignon de Oliveira

Nome: Bruno Marques Li Volsi Falcao

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [134]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [135]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Owner\Desktop\CDados\Cdados-aps4


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [136]:
filename = 'Ubisoft.entrega.xlsx'

In [137]:
treino = pd.read_excel(filename)
treino.head(5)


,Treinamento,Relevância
0,"mano, pq o recoil sempre vem uma merda ? pqp u...",1
1,"toma meu dinheiro epic e ubisoft, conseguiram ...",0
2,a @ubisoft lançou a season nova ontem e so col...,1
3,16 reais mascando na steam serei obrigado a da...,0
4,@kamilessie fui la uma vez kkkkk patinar no ge...,0


In [138]:
teste = pd.read_excel(filename, sheet_name = 'Teste')
teste.head(5)

,Teste,Relevância
0,incrível como até o site da ubisoft é horroroso,0.0
1,@pastorxbox vem pra nois ubisoft ai você termi...,0.0
2,comunicado oficial da @ubisoft a respeito do p...,1.0
3,@slapyr6 @ubisoft reinicia o jogo,0.0
4,pq a ubisoft ainda existe? https://t.co/gmruwr...,0.0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Consideramos os comentarios em que a pessoa deixa alguma reclamação ou dizendo algum problema que teve com a marca como relevantes, justamente, para a empresa poder ver no que melhorar. 

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [139]:
# Função que faz a Limpeza de Strings
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;,"”“)(/|[\]]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

    

In [140]:
treino

,Treinamento,Relevância
0,"mano, pq o recoil sempre vem uma merda ? pqp u...",1
1,"toma meu dinheiro epic e ubisoft, conseguiram ...",0
2,a @ubisoft lançou a season nova ontem e so col...,1
3,16 reais mascando na steam serei obrigado a da...,0
4,@kamilessie fui la uma vez kkkkk patinar no ge...,0
...,...,...
295,rt @savageymir: assassin ⚔ #likeanassassin\n\n...,0
296,@eleze_pokas é a segunda vez que trava\nubisof...,1
297,todos os jogos da ubisoft desde 2015 https://t...,0
298,@_abovee_ envia pra gente aqui\nhttps://t.co/e...,0


In [141]:
# Coloca um documento em uma lista
tweetsR = treino.loc[treino.Relevância == 1,"Treinamento"]
tweetsI = treino.loc[treino.Relevância == 0,"Treinamento"]
#Cria uma Coluna nova
teste['Classificação']=''
teste

,Teste,Relevância,Classificação
0,incrível como até o site da ubisoft é horroroso,0.0,
1,@pastorxbox vem pra nois ubisoft ai você termi...,0.0,
2,comunicado oficial da @ubisoft a respeito do p...,1.0,
3,@slapyr6 @ubisoft reinicia o jogo,0.0,
4,pq a ubisoft ainda existe? https://t.co/gmruwr...,0.0,
...,...,...,...
322,o #br6 começa no próximo sábado (20).\n\nhttps...,NaN,
323,@xinok666 @pomo_d_oro @rato_sensivel se tem mu...,NaN,
324,rt @pastorxbox: vazou um teste para um jogo nã...,NaN,
325,@gcgameroficial queria saber pq a epic e a ubi...,NaN,


In [142]:
#Limpar cada frase

R = []
for i in tweetsR:
    R.append(cleanup(i))
R = " ".join(R).split() 

I = []
for i in tweetsI:
    I.append(cleanup(i))
I = " ".join(I).split()

T = []
for i in treino.Treinamento:
    T.append(cleanup(i))
T = " ".join(T).split()

In [143]:
# pegar os valores relativos de cada palavra 
total= pd.Series(T)
total_a= total.value_counts()
total_r= total.value_counts(True)

relevante = pd.Series(R)
relevante_a = relevante.value_counts() 
relevante_r = relevante.value_counts(True)

irrelevante = pd.Series(I)
irrelevante_a = irrelevante.value_counts() 
irrelevante_r = irrelevante.value_counts(True)

In [144]:
#Conta de probabilidade de ser um ou outro
probR = sum(relevante_a)/sum(total_a)

print(probR)

probI = sum(irrelevante_a)/sum(total_a)

print(probI)


0.4010791366906475
0.5989208633093526


In [145]:
TT = []
for i in teste.Teste:
    TT.append(cleanup(i).split())

In [146]:
#Classificador Naives Bayes
contador= 0
l_R= []
l_I= []
p= []
for n in TT:
    
    probabilidadeR= 1
    for c in n:
        if c in relevante_a:
            a= (relevante_a[c] + 1) / (len(relevante) + len(total_a))
            probabilidadeR= a * probabilidadeR
        else:
            a= 1 / (len(relevante) + len(total_a))
            probabilidadeR= a * probabilidadeR
            
    probabilidadeI= 1
    for c in n:
        if c in irrelevante_a:
            a= (irrelevante_a[c] + 1) / (len(irrelevante) + len(total_a))
            probabilidadeI= probabilidadeI * a
        else:
            a= 1 / (len(irrelevante) + len(total_a))
            probabilidadeI= probabilidadeI * a
    probabilidadeFR = probabilidadeR * probR
    
    probabilidadeFI = probabilidadeI * probI
    

    if probabilidadeFR>probabilidadeFI: 
        teste.iloc[contador,2]= 'Relevante'
            

    elif probabilidadeFR<probabilidadeFI: 
        teste.iloc[contador,2]= 'Irrelevante'

    contador+= 1

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [147]:

teste.head(20)

,Teste,Relevância,Classificação
0,incrível como até o site da ubisoft é horroroso,0.0,Irrelevante
1,@pastorxbox vem pra nois ubisoft ai você termi...,0.0,Irrelevante
2,comunicado oficial da @ubisoft a respeito do p...,1.0,Relevante
3,@slapyr6 @ubisoft reinicia o jogo,0.0,Irrelevante
4,pq a ubisoft ainda existe? https://t.co/gmruwr...,0.0,Irrelevante
5,@overvghoul @eddyderisio é porque esse é da ub...,0.0,Irrelevante
6,rt @bardinhoh: isso que eu chamo de sequência ...,0.0,Irrelevante
7,comprei the crew e trackmania ontem. mas tô ac...,1.0,Relevante
8,@johnmsrton isso é a ubi que faz em todo jogo ...,1.0,Irrelevante
9,cara eu nao quero parar mas a ubisoft parece q...,1.0,Relevante


In [148]:
teste['Resultado']= ''
for i in range(len(teste)):
    if teste.iloc[i,1] == 1 and teste.iloc[i,2] =='Relevante':
        teste.iloc[i,3] = 'Verdadeiro Positivo'
    elif teste.iloc[i,1] == 0 and teste.iloc[i,2] =='Irrelevante':
        teste.iloc[i,3] = 'Verdadeiro Negativo'
    elif teste.iloc[i,1] == 0 and teste.iloc[i,2] =='Relevante':
        teste.iloc[i,3] = 'Falso Positivo'
    elif teste.iloc[i,1] == 1 and teste.iloc[i,2] =='Irrelevante':
        teste.iloc[i,3] = 'Falso Negativo'


In [149]:
teste.head(20)

,Teste,Relevância,Classificação,Resultado
0,incrível como até o site da ubisoft é horroroso,0.0,Irrelevante,Verdadeiro Negativo
1,@pastorxbox vem pra nois ubisoft ai você termi...,0.0,Irrelevante,Verdadeiro Negativo
2,comunicado oficial da @ubisoft a respeito do p...,1.0,Relevante,Verdadeiro Positivo
3,@slapyr6 @ubisoft reinicia o jogo,0.0,Irrelevante,Verdadeiro Negativo
4,pq a ubisoft ainda existe? https://t.co/gmruwr...,0.0,Irrelevante,Verdadeiro Negativo
5,@overvghoul @eddyderisio é porque esse é da ub...,0.0,Irrelevante,Verdadeiro Negativo
6,rt @bardinhoh: isso que eu chamo de sequência ...,0.0,Irrelevante,Verdadeiro Negativo
7,comprei the crew e trackmania ontem. mas tô ac...,1.0,Relevante,Verdadeiro Positivo
8,@johnmsrton isso é a ubi que faz em todo jogo ...,1.0,Irrelevante,Falso Negativo
9,cara eu nao quero parar mas a ubisoft parece q...,1.0,Relevante,Verdadeiro Positivo


___
### Concluindo

5b) As mensagens que tem dupla negação e sarcasmo são contadas igual todas as outras porque elas não são consideradas como diferencias .

In [150]:
Proporção = pd.crosstab(teste["Classificação"],teste['Relevância'], normalize=True).round(6)*100
Proporção

Relevância,0.0,1.0
Classificação,,
Irrelevante,41.7085,9.5477
Relevante,24.1206,24.6231


5c) Um plano de expansão viável para esse projeto seria a limpeza e separaçao de emojis, e um filtrador mais preciso, no qual, classifica ao invés de duas categorias, poderia classificar em 4, como: 'Relevante', 'muito Relevante', 'Irrelevante', 'muito Irrelevante'.

A Ubisoft deveria continuar financiando esse projeto para ter um melhor desempenho na criação de produtos a partir de critícas construtivas vistas no twitter (uma plataforma de grande importância de troca de informação), consequentemente, melhorando seus produtos.

 classificador Naive Bayes em diferentes contextos:

Classificador Naive Bayes pode ser útil em outros contextos. Por exemplo, esse método pode ser usado para fazendas de pecuária de corte em torno do nosso país, verificando qual a quantidade de gordura entremeada na carne do gado classificando-as em 12 níveis(grau) de marmoreio, caracterizando-as de ruim a excelente de acordo com o grau de marmoreio. Obviamente seria mais complexo e teria variás outras melhorias, tendo contar o numero de Açougues que vendem esse tipo de carne, conectados digitalmente com a base de dados.

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**